In [2]:
#importing libraries
import pandas as pd
import numpy as np
import os
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
!pip install datasets
from datasets import Dataset, DatasetDict
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from transformers import EvalPrediction

/Users/yasamanmajedi/.local/share/virtualenvs/yasamanmajedi-llyTPa9M/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [3]:
file_path = "data/EN/subtask-2-annotations.txt"
df = pd.read_csv(file_path, sep="\t", header=None, names=["Document_ID", "High_Level_Narratives", "Sub_Narratives"])

In [4]:
df

,Document_ID,High_Level_Narratives,Sub_Narratives
0,EN_CC_100013.txt,CC: Criticism of climate movement,CC: Criticism of climate movement: Ad hominem ...
1,EN_UA_300009.txt,Other,Other
2,EN_UA_300017.txt,Other,Other
3,EN_CC_100021.txt,Other,Other
4,EN_UA_300041.txt,Other,Other
...,...,...,...
394,EN_CC_200022.txt,CC: Criticism of institutions and authorities;...,CC: Criticism of institutions and authorities:...
395,EN_CC_100028.txt,Other,Other
396,EN_CC_300010.txt,CC: Amplifying Climate Fears,CC: Amplifying Climate Fears: Other
397,EN_UA_013257.txt,URW: Russia is the Victim;URW: Blaming the war...,URW: Russia is the Victim: Russia actions in U...


In [6]:
df = df.groupby("Document_ID").agg({
    "High_Level_Narratives": lambda x: list(set(";".join(x).split(";"))),
    "Sub_Narratives": lambda x: list(set(";".join(x).split(";")))
}).reset_index()

df.rename(columns={
    "High_Level_Narratives": "High_Level_Narratives_List",
    "Sub_Narratives": "Sub_Narratives_List"
}, inplace=True)

df

,Document_ID,High_Level_Narratives_List,Sub_Narratives_List
0,EN_CC_100000.txt,[CC: Hidden plots by secret schemes of powerfu...,[CC: Criticism of institutions and authorities...
1,EN_CC_100002.txt,[CC: Hidden plots by secret schemes of powerfu...,[CC: Criticism of institutions and authorities...
2,EN_CC_100003.txt,[CC: Criticism of institutions and authorities...,[CC: Criticism of climate movement: Ad hominem...
3,EN_CC_100004.txt,"[CC: Controversy about green technologies, CC:...",[CC: Criticism of climate movement: Climate mo...
4,EN_CC_100005.txt,[CC: Criticism of climate movement],"[CC: Criticism of climate movement: Other, CC:..."
...,...,...,...
394,EN_UA_DEV_100028.txt,[URW: Negative Consequences for the West],[URW: Negative Consequences for the West: Othe...
395,EN_UA_DEV_216.txt,"[URW: Negative Consequences for the West, URW:...","[URW: Discrediting the West, Diplomacy: The EU..."
396,EN_UA_DEV_23.txt,"[URW: Amplifying war-related fears, URW: Prais...",[URW: Distrust towards Media: Western media is...
397,EN_UA_DEV_24.txt,"[URW: Negative Consequences for the West, URW:...","[URW: Discrediting the West, Diplomacy: Diplom..."


In [7]:

documents_folder = 'data/EN/raw-documents'


def load_documents(folder_path):
    documents = {}
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):  # Ensure only text files are read
            doc_id = filename  # Extract Document_ID from filename
            with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as file:
                documents[doc_id] = file.read().strip()
    return documents

# Load document texts into a dictionary
document_texts = load_documents(documents_folder)

# Map document texts to the DataFrame
df['Text'] = df['Document_ID'].map(document_texts)

# Check if all documents are successfully mapped
print(f"Number of missing documents: {df['Text'].isnull().sum()}")
df


Number of missing documents: 0


,Document_ID,High_Level_Narratives_List,Sub_Narratives_List,Text
0,EN_CC_100000.txt,[CC: Hidden plots by secret schemes of powerfu...,[CC: Criticism of institutions and authorities...,Pentagon plans to serve LAB-GROWN MEAT to troo...
1,EN_CC_100002.txt,[CC: Hidden plots by secret schemes of powerfu...,[CC: Criticism of institutions and authorities...,Oxford Residents Mount Resistance Against the ...
2,EN_CC_100003.txt,[CC: Criticism of institutions and authorities...,[CC: Criticism of climate movement: Ad hominem...,"Fonda Heads To Canada For Oil Sands Protest, M..."
3,EN_CC_100004.txt,"[CC: Controversy about green technologies, CC:...",[CC: Criticism of climate movement: Climate mo...,A Tesla Owner Just Exposed A Sick Secret About...
4,EN_CC_100005.txt,[CC: Criticism of climate movement],"[CC: Criticism of climate movement: Other, CC:...",Climate Crazies Fail in Attempt to Vandalize A...
...,...,...,...,...
394,EN_UA_DEV_100028.txt,[URW: Negative Consequences for the West],[URW: Negative Consequences for the West: Othe...,European gas prices surge 20% as Russia's late...
395,EN_UA_DEV_216.txt,"[URW: Negative Consequences for the West, URW:...","[URW: Discrediting the West, Diplomacy: The EU...","EU 'biggest loser' in Ukraine conflicts, Hunga..."
396,EN_UA_DEV_23.txt,"[URW: Amplifying war-related fears, URW: Prais...",[URW: Distrust towards Media: Western media is...,What is the current trajectory of the evil emp...
397,EN_UA_DEV_24.txt,"[URW: Negative Consequences for the West, URW:...","[URW: Discrediting the West, Diplomacy: Diplom...",Europe ‘Shot Itself in the Lungs’ With Sanctio...


In [8]:
urw_narratives = [
    "URW: Amplifying war-related fears",
    "URW: Blaming the war on others rather than the invader",
    "URW: Discrediting Ukraine",
    "URW: Discrediting the West, Diplomacy",
    "URW: Distrust towards Media",
    "URW: Hidden plots by secret schemes of powerful groups",
    "URW: Negative Consequences for the West",
    "URW: Overpraising the West",
    "URW: Praise of Russia",
    "URW: Russia is the Victim",
    "URW: Speculating war outcomes",
]

In [11]:
df = df[df["High_Level_Narratives_List"].apply(lambda narratives: any("URW:" in narrative for narrative in narratives))]


In [16]:
def preprocess_multi_label(data, narratives):
    label_vectors = []
    for narratives_list in data['High_Level_Narratives_List']:
        vector = [1 if narrative in narratives_list else 0 for narrative in narratives]
        label_vectors.append(vector)
    data['Labels'] = label_vectors
    return data

In [17]:
df = preprocess_multi_label(df, urw_narratives)

/var/folders/l4/jfvbkqp14fd8m7dlf36rj__r0000gn/T/ipykernel_3100/2821491017.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Labels'] = label_vectors


In [18]:
def calculate_class_weights(labels):
    label_sums = np.sum(labels, axis=0)
    total_samples = len(labels)
    class_weights = [total_samples / (len(label_sums) * count) for count in label_sums]
    return torch.tensor(class_weights, dtype=torch.float)

In [19]:
labels = np.array(df['Labels'].tolist(), dtype=np.float32)
class_weights = calculate_class_weights(labels)

In [20]:
def handle_rare_classes(data, labels, min_samples=15):
    label_sums = np.sum(labels, axis=0)
    rare_classes = np.where(label_sums < min_samples)[0]

    for rare_class in rare_classes:
        rare_indices = [i for i, lbl in enumerate(labels) if lbl[rare_class] == 1]
        if len(rare_indices) > 0:
            duplicate_data = data.iloc[rare_indices]
            data = pd.concat([data] + [duplicate_data] * (min_samples - len(rare_indices)), ignore_index=True)
            labels = np.vstack([labels] + [labels[rare_indices]] * (min_samples - len(rare_indices)))
    return data, labels


In [21]:
df, labels = handle_rare_classes(df, labels)

In [22]:
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['Text'], labels, test_size=0.2, random_state=42)


In [23]:
train_dataset = Dataset.from_dict({"text": train_texts.tolist(), "labels": train_labels.tolist()})
test_dataset = Dataset.from_dict({"text": test_texts.tolist(), "labels": test_labels.tolist()})
datasets = DatasetDict({"train": train_dataset, "test": test_dataset})

In [24]:
tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")
model = AutoModelForSequenceClassification.from_pretrained(
    "roberta-base", num_labels=labels.shape[1], problem_type="multi_label_classification"
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [25]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

# Tokenize the dataset
tokenized_datasets = datasets.map(tokenize_function, batched=True)


Map: 100%|██████████| 43/43 [00:00<00:00, 1201.59 examples/s]


In [17]:
# def custom_loss_function(outputs, targets):
#     sigmoid = torch.nn.Sigmoid()
#     bce = torch.nn.BCEWithLogitsLoss(weight=class_weights)
#     return bce(outputs, targets)

In [26]:
from sklearn.metrics import precision_score, recall_score, f1_score
from transformers import EvalPrediction

def compute_metrics(p: EvalPrediction):
    # Apply sigmoid activation to model predictions (logits)
    sigmoid_preds = torch.sigmoid(torch.tensor(p.predictions)).numpy()

    # Convert probabilities to binary predictions using a threshold of 0.5
    preds = (sigmoid_preds > 0.5).astype(int)

    # Ground-truth labels
    labels = p.label_ids

    # Calculate weighted precision, recall, and F1-score
    precision = precision_score(labels, preds, average="weighted", zero_division=0)
    recall = recall_score(labels, preds, average="weighted", zero_division=0)
    f1 = f1_score(labels, preds, average="weighted", zero_division=0)

    # Return the computed metrics
    return {"precision": precision, "recall": recall, "f1": f1}


In [27]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [28]:

training_args = TrainingArguments(
    output_dir='./results',                    # Directory to store checkpoints and final model
    num_train_epochs=10,                       # Total number of training epochs
    learning_rate=2e-5,  # Fine-tuning learning rate
    per_device_train_batch_size=4,            # Batch size per device during training
    per_device_eval_batch_size=4,             # Batch size for evaluation
    evaluation_strategy='epoch',               # Evaluate at the end of each epoch
    save_strategy='epoch',                     # Save model at the end of each epoch
    load_best_model_at_end=True,               # Load the best model at the end of training
    metric_for_best_model='f1',                # Use F1 score to evaluate the best model
    greater_is_better=True,                    # Higher F1 is better
    logging_dir='./logs',                      # Directory for storing logs
    logging_steps=100,                         # Log every 100 steps
    save_total_limit=3,                        # Limit the total amount of checkpoints
    seed=42,
    weight_decay=0.01,# Seed for reproducibility
    fp16=True
)

/Users/yasamanmajedi/.local/share/virtualenvs/yasamanmajedi-llyTPa9M/lib/python3.10/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [29]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/var/folders/l4/jfvbkqp14fd8m7dlf36rj__r0000gn/T/ipykernel_3100/3137798730.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


ValueError: fp16 mixed precision requires a GPU (not 'mps').

In [22]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,No log,0.467593,0.000000,0.000000,0.000000
2,No log,0.415626,0.384414,0.224490,0.276183
3,0.492600,0.385270,0.433302,0.275510,0.322723
4,0.492600,0.354353,0.511774,0.397959,0.440743
5,0.380900,0.328297,0.727911,0.510204,0.563719
6,0.380900,0.316760,0.729364,0.571429,0.595059
7,0.319600,0.300169,0.776811,0.561224,0.623142
8,0.319600,0.288163,0.862185,0.622449,0.687935
9,0.319600,0.280087,0.947279,0.653061,0.747331
10,0.277800,0.279120,0.900227,0.663265,0.736958


TrainOutput(global_step=430, training_loss=0.36054611649624135, metrics={'train_runtime': 227.3813, 'train_samples_per_second': 7.52, 'train_steps_per_second': 1.891, 'total_flos': 449956261509120.0, 'train_loss': 0.36054611649624135, 'epoch': 10.0})

In [23]:
def predict_with_threshold(trainer, dataset, threshold=0.7):
    predictions = trainer.predict(dataset)
    logits = predictions.predictions
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.tensor(logits))
    return (probs > threshold).int()

# Predict and return results
results = predict_with_threshold(trainer, tokenized_datasets["test"])
print("Predicted Labels:", results)

Predicted Labels: tensor([[0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        [1, 0, 0, 1, 0, 0, 0, 1, 0, 